In [49]:
from typing import TypedDict, Annotated, Literal

from langchain_openai  import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import  StateGraph, END, add_messages
from langgraph.prebuilt import  ToolNode


In [50]:
model = ChatOpenAI(
    base_url="https://ws-02.wade0426.me/v1",
    api_key="day3hw",
    model="Qwen/Qwen3-VL-8B-Instruct",
    max_tokens=50,
    temperature=0
)

In [51]:
@tool
def extract_order_data(name: str, phone: str, product: str, quantity: int, address: str):
    """
    資料提取專用工具
    專門用於從非結構話文本中提取訂單相關資訊（姓名，電話，商品，數量，地址）。
    """

    return {
        "name": name,
        "phone": phone,
        "product": product,
        "quantity": quantity,
        "address": address
    }

llm_with_tools = model.bind_tools([extract_order_data])

In [52]:
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

def call_model(state: AgentState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode([extract_order_data])

def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [53]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", END: END}
)

workflow.add_edge("tools", "agent")

In [54]:
app = workflow.compile()
print(app.get_graph().draw_ascii())

if __name__ == "__main__":
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "q"]: break

        for event in app.stream({"messages": [HumanMessage(content=user_input)]}):
            for key, value in event.items():
                print(f"\n --- Node: {key} ---")
                print(value["messages"][-1].content or value["messages"][-1].tool_calls)

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +-------+           
          | agent |           
          +-------+.          
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 

 --- Node: agent ---
您好！請問有什麼我可以幫忙的嗎？

 --- Node: agent ---
[{'name': 'extract_order_data', 'args': {'name': '陳大明', 'phone': '0912-345-678', 'product': '筆記型電腦', 'quantity': 3, 'address': '台中市北區'}, 'id': 'MEyMvVw7yhsa8M5QTbAL8VyGgQb45SgE', 'type': 'tool_call'}]

 --- Node: tools ---
{"name": "陳大明", "phone": "0912-345-678", "product": "筆記型電腦", "quantity": 3, "address": "台中市北區"}

 --- Node: agent ---
好的，陳大明先生，您的訂單已確認：3台筆記型電腦，將於下週五送到台中市北區。您的聯絡電話是0912-345-678。


KeyboardInterrupt: Interrupted by user

In [55]:
@tool
def get_weather(city: str):
    """查詢指定城市的天氣。輸入參數 city 必須是城市名稱。"""
    if "台北" in city:
        return "台北下大雨，氣溫 18 度"
    elif "台中" in city:
        return "台中天晴，氣溫 26 度"
    elif "高雄" in city:
        return "高雄多雲，氣溫 30 度"
    else:
        return "資料庫沒有這個程式的資料"


In [56]:
tools = [get_weather]
llm_with_tools = model.bind_tools(tools)

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


def chatbot_node(state: AgentState):
    """思考節點：負責呼叫 LLM"""
    messages = state["messages"]
    response = llm_with_tools.invoke(state["messages"])

    return {"messages": [response]}

In [57]:
tool_node_executor = ToolNode(tools)

def router(state: AgentState) -> Literal["tools", "end"]:
    """路由邏輯： 決定下一步是執行工具還是結束"""
    messages = state["messages"]
    last_messages = messages[-1]

    if last_messages.tool_calls:
        return "tools"
    else:
        return "end"

In [58]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", chatbot_node)
workflow.add_node("tools", tool_node_executor)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    router,
    {
        "tools": "tools",
        END: END
    }
)

In [59]:
workflow.add_edge("tools", "agent")

app = workflow.compile()
print(app.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +-------+           
          | agent |           
          +-------+*          
          .         *         
        ..           **       
       .               *      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 
